In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import numpy as np

C:\Users\prit4\AppData\Local\Temp\ipykernel_19964\291067091.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
dataset_path = "dataset.csv"
df = pd.read_csv(dataset_path)

In [3]:
# Remove negative values from the dataset
for column in df.select_dtypes(include=['int64', 'float64']).columns:
    df = df[df[column] >= 0]

print('Negative values removed from the dataset.')

Negative values removed from the dataset.


In [4]:
scaler = MinMaxScaler()
scaled_features = scaler.fit_transform(df[['month', 'day', 'time', 'humidity', 'tempC']])

In [5]:
X = scaled_features
y = df['fan_speed'].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)


In [6]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(50, activation='relu', input_shape=(5,)),
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dense(6, activation='softmax')  # Softmax activation for multi-class classification
])

In [7]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Train the model
history = model.fit(X_train, y_train, epochs=100, batch_size=64, validation_split=0.2)



Epoch 1/100


88/88 [==============================] - 3s 9ms/step - loss: 1.5078 - accuracy: 0.3885 - val_loss: 1.3652 - val_accuracy: 0.4147
Epoch 2/100
88/88 [==============================] - 0s 4ms/step - loss: 1.2811 - accuracy: 0.4551 - val_loss: 1.1770 - val_accuracy: 0.5270
Epoch 3/100
88/88 [==============================] - 0s 4ms/step - loss: 1.1335 - accuracy: 0.5204 - val_loss: 1.0560 - val_accuracy: 0.5562
Epoch 4/100
88/88 [==============================] - 0s 5ms/step - loss: 1.0234 - accuracy: 0.5656 - val_loss: 0.9536 - val_accuracy: 0.6081
Epoch 5/100
88/88 [==============================] - 0s 4ms/step - loss: 0.9336 - accuracy: 0.6058 - val_loss: 0.8735 - val_accuracy: 0.6415
Epoch 6/100
88/88 [==============================] - 0s 4ms/step - loss: 0.8614 - accuracy: 0.6399 - val_loss: 0.8128 - val_accuracy: 0.6693
Epoch 7/100
88/88 [==============================] - 0s 4ms/step - loss: 0.8002 - accuracy: 0.6703 - val_loss: 0.7546 - val_accuracy: 0.6728
Epoch 8/10

In [16]:
# Convert true classes to one-hot encoding
true_classes_one_hot = tf.keras.utils.to_categorical(y_test, num_classes=6)
predicted_classes = model.predict(X_test)
# print(predicted_classes)
# Calculate accuracy
accuracy = np.mean(np.argmax(predicted_classes, axis=1) == np.argmax(true_classes_one_hot, axis=1))
print("Test Accuracy (Threshold):", accuracy)


55/55 [==============================] - 0s 2ms/step
[[1.10508743e-27 2.46476971e-11 1.31704565e-02 9.51517403e-01
  3.53120193e-02 1.42769721e-07]
 [4.04543564e-17 2.45937326e-06 3.69454265e-01 6.30282998e-01
  2.60231463e-04 4.65388526e-11]
 [0.00000000e+00 1.04500530e-23 8.02778011e-10 3.28547409e-04
  9.17351127e-01 8.23203921e-02]
 ...
 [2.78806596e-07 2.52183110e-01 7.47483492e-01 3.33108852e-04
  2.74766798e-12 2.86797089e-20]
 [2.71444762e-04 9.17863488e-01 8.18647817e-02 2.00783717e-07
  6.08736374e-16 1.36624603e-23]
 [0.00000000e+00 1.54327127e-30 3.50565592e-14 3.52952611e-07
  3.63356061e-02 9.63664055e-01]]
Test Accuracy (Threshold): 0.8935685828116107


In [15]:
model.save('model.h5')

c:\Users\prit4\OneDrive\Desktop\stuff\active_Github_repos\Mini_project\myenv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
